### Lec 09-1: XOR문제 딥러닝으로 풀기

 - cf. (복습) sigmoid 함수 : x값이 커질수록 1에, 작아질수록 0에 근접한 값을 가진다.
 
 <left><img src= 'img/sigmoid.PNG' width="50%"></left>

- logistic regression 한 개만 사용해서는 XOR 문제를 해결할 수 없지만, 그림처럼 3개를 연결해서 사용한다면 가능하다. 
- 이 부분이 Neural Network의 핵심일 수 있다. 기존의 방식을 연결해서 사용! 

<left><img src= 'img/xor_2.PNG' width="40%"></left>
<left><img src= 'img/xor_3.PNG' width="31%"></left>

- 위 3개의 logistic을 합쳐서 Neural Network(NN)을 구성할 수 있다. 
- 이것을 다시 matrix를 적용하여 우측 그림과 같이 간단히 표현할 수 있다. 

<left><img src= 'img/nn.PNG' width="50%"></left>

- 코딩으로 표현하면 아래와 같다.  
- 마지막 수식이 k가 되고, 여기에 다시 sigmoid를 적용한 함수가 가설 H(x)가 된다. 

<left><img src= 'img/nn2.PNG' width="50%"></left>

### Lec 09-x: 10분안에 미분 정리 

- 미분은 순간변화량, 즉 기울기를 구하는 것이다.
- <font color = 'yellow'> 상수(constant value)에 대한 미분값은 항상 0이다. </font>

<left><img src= 'img/derivative.PNG' width="50%"></left>

- 편미분은 두 변수 중 하나에 대해서만 미분하고, 나머지 변수는 상수로 취급한다는 것이다. 

<left><img src= 'img/p_derivative.PNG' width="31%"></left>
<left><img src= 'img/p_derivative2.PNG' width="30%"></left>

### Lec 09-2: 딥넷트웍 학습 시키기

- Neural Network의 node가 증가함에 따라 미분계산량이 급증, 현실적으로 계산해내기 어려웠다. 
- 계산결과를 실제값과 비교하여, 이를 다시 Backward Propagation하여 W를 재조정 하는 방식이 고안되었다. 

<left><img src= 'img/back_propagation.PNG' width="50%"></left>

- forward 값은 training data로 부터 가져온 실제 값이다. 
- 편미분을 통해 w, x, b의 미분값(기울기, 즉 변화량이 결과값에 미치는 영향도)을 구할 수 있다. 
- <font color = 'yellow'> 이처럼 알고있는 값으로 부터 시작해 거꾸로 미분을 수행하는 것이 backword propagation이다. </font>

<left><img src= 'img/chain_rule.PNG' width="50%"></left>

- 같은 방식을 여러개의 노드에 반복함으로써, Neural Network의 계산이 이루어진다. 

<left><img src= 'img/chain_rule2.PNG' width="50%"></left>

### Lab 09-1: Neural net for XOR

In [1]:
#### xor 문제를 tf로 풀어보기 
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
sess = tf.InteractiveSession()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
#### XOR 표를 데이터 형태로 만들어 읽어온다. 
x_data = np.array([ [0,0], [0,1], [1,0], [1,1] ], dtype = np.float32)
y_data = np.array([ [0],   [1],   [1],   [0] ],   dtype = np.float32)
print(x_data)
print(y_data)

[[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]]
[[0.]
 [1.]
 [1.]
 [0.]]


In [3]:
#### logistic으로 xor을 풀어보면 어떨까? 
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)
W = tf.Variable(tf.random_normal([2,1]), name = 'weight')
b = tf.Variable(tf.random_normal([1]), name = 'bias')

hypothesis = tf.sigmoid(tf.matmul(X,W)+b)
cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y)*tf.log(1-hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

#### hypothesis > 0.5 => True 
predicted = tf.cast(hypothesis > 0.5, dtype = tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype = tf.float32))

In [4]:
#### graphing 
with tf.Session() as sess : 
    sess.run(tf.global_variables_initializer())
    for step in range(10001) : 
        sess.run(train, feed_dict = {X: x_data, Y: y_data})
        if step %2000 == 0 : 
            print(step, sess.run(cost, feed_dict = {X: x_data, Y: y_data}), sess.run(W))
            #### accuracy report 
            h,c,a = sess.run([hypothesis, predicted, accuracy], feed_dict = {X: x_data, Y: y_data})
            print('\n >Hypothesis :', h, 
                  '\n >Correct :', c, 
                  '\n >Accuracy :', a)

0 0.9006585 [[0.9673923]
 [2.4748657]]

 >Hypothesis : [[0.11898362]
 [0.6160404 ]
 [0.26217428]
 [0.8084806 ]] 
 >Correct : [[0.]
 [1.]
 [0.]
 [1.]] 
 >Accuracy : 0.5
2000 0.69314724 [[0.00074925]
 [0.00075491]]

 >Hypothesis : [[0.499777  ]
 [0.49996567]
 [0.4999643 ]
 [0.500153  ]] 
 >Correct : [[0.]
 [0.]
 [0.]
 [1.]] 
 >Accuracy : 0.25
4000 0.6931472 [[2.6708702e-07]
 [2.6609246e-07]]

 >Hypothesis : [[0.49999988]
 [0.5       ]
 [0.5       ]
 [0.50000006]] 
 >Correct : [[0.]
 [0.]
 [0.]
 [1.]] 
 >Accuracy : 0.25
6000 0.6931472 [[1.3297606e-07]
 [1.3198149e-07]]

 >Hypothesis : [[0.5]
 [0.5]
 [0.5]
 [0.5]] 
 >Correct : [[0.]
 [0.]
 [0.]
 [0.]] 
 >Accuracy : 0.5
8000 0.6931472 [[1.3297606e-07]
 [1.3198149e-07]]

 >Hypothesis : [[0.5]
 [0.5]
 [0.5]
 [0.5]] 
 >Correct : [[0.]
 [0.]
 [0.]
 [0.]] 
 >Accuracy : 0.5
10000 0.6931472 [[1.3297606e-07]
 [1.3198149e-07]]

 >Hypothesis : [[0.5]
 [0.5]
 [0.5]
 [0.5]] 
 >Correct : [[0.]
 [0.]
 [0.]
 [0.]] 
 >Accuracy : 0.5


- XOR 문제에 대해 logistic의 정확도는 0.5밖에 나오지 않았다
- XOR 문제를 풀기 위해서는 Neural Network가 필요하다. 

- <font color = 'yellow'> 두개의 layer를 구성하면 된다. </font>

<left><img src= 'img/nn.PNG' width="50%"></left>

In [5]:
x_data = np.array([ [0,0], [0,1], [1,0], [1,1] ], dtype = np.float32)
y_data = np.array([ [0],   [1],   [1],   [0] ],   dtype = np.float32)

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

#### 두개의 layer를 가지는 nn을 구성하고, 이로부터 가설을 만든다 #######################
#### layer1은 출력값이 2개임에 주의!!
W1 = tf.Variable(tf.random_normal([2,2]), name = 'weight1')
b1 = tf.Variable(tf.random_normal([2]), name = 'bias1')
layer1 = tf.sigmoid(tf.matmul(X,W1)+b1)

W2 = tf.Variable(tf.random_normal([2,1]), name = 'weight2')
b2 = tf.Variable(tf.random_normal([2]), name = 'bias2')

hypothesis = tf.sigmoid(tf.matmul(layer1,W2)+b2)
########################################################################################

cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y)*tf.log(1-hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

#### hypothesis > 0.5 => True 
predicted = tf.cast(hypothesis > 0.5, dtype = tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype = tf.float32))

#### graphing 
with tf.Session() as sess : 
    sess.run(tf.global_variables_initializer())
    for step in range(10001) : 
        sess.run(train, feed_dict = {X: x_data, Y: y_data})
        if step %2000 == 0 : 
            print(step, sess.run(cost, feed_dict = {X: x_data, Y: y_data}), sess.run(W))
            #### accuracy report 
            h,c,a = sess.run([hypothesis, predicted, accuracy], feed_dict = {X: x_data, Y: y_data})
            print('\n >Hypothesis :', h, 
                  '\n >Correct :', c, 
                  '\n >Accuracy :', a)

0 0.93360746 [[0.41870815]
 [0.6592225 ]]

 >Hypothesis : [[0.10637787 0.32830006]
 [0.14445806 0.4094247 ]
 [0.10688201 0.32946822]
 [0.16724578 0.45193085]] 
 >Correct : [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]] 
 >Accuracy : 0.5
2000 0.44096738 [[0.41870815]
 [0.6592225 ]]

 >Hypothesis : [[0.12406418 0.12406418]
 [0.6243787  0.6243787 ]
 [0.64046335 0.64046335]
 [0.51073223 0.51073223]] 
 >Correct : [[0. 0.]
 [1. 1.]
 [1. 1.]
 [1. 1.]] 
 >Accuracy : 0.75
4000 0.081854045 [[0.41870815]
 [0.6592225 ]]

 >Hypothesis : [[0.04733804 0.04733804]
 [0.9105731  0.9105731 ]
 [0.9102638  0.9102638 ]
 [0.08718234 0.08718234]] 
 >Correct : [[0. 0.]
 [1. 1.]
 [1. 1.]
 [0. 0.]] 
 >Accuracy : 1.0
6000 0.03514751 [[0.41870815]
 [0.6592225 ]]

 >Hypothesis : [[0.02393755 0.02393755]
 [0.96012276 0.96012276]
 [0.96002114 0.96002114]
 [0.03426635 0.03426635]] 
 >Correct : [[0. 0.]
 [1. 1.]
 [1. 1.]
 [0. 0.]] 
 >Accuracy : 1.0
8000 0.021771602 [[0.41870815]
 [0.6592225 ]]

 >Hypothesis : [[0.01562652 0.0156

- NN 결과 100%의 정확도로 xor 문제를 풀어내었다. 
- <font color = 'yellow'> 처음 입력값의 갯수(여기서는 2)와 마지막 출력값의 갯수(여기서는 1)만 틀리지 않으면, 모형을 얼마든지 wide & deep 하게 만들수 있다. 
  (앞단계의 출력값 = 그 다음단계의 입력값)</font>

<left><img src= 'img/deepnn.PNG' width="55%"></left>

### Lab 09-2: Tensor board 

- 1. 텐서플로우에 loging할 값을 결정한다. 
- 2. 값들을 모두 합친다. 
- 3. 합쳐진 값을 저장할 위치를 정하고, 세션에 그래프를 넣는다. 
- 4. 세션을 실행하고, 파일에 기록한다.  
- 5. tensorboard를 실행하고, 기록된 파일의 위치를 알려준다. 

<left><img src= 'img/five_steps.PNG' width="45%"></left>

In [6]:
#### MNIST를 tensorboard에 적용해 보자 (tf 2.0 code)

import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2], name="x")
Y = tf.placeholder(tf.float32, [None, 1], name="y")

with tf.name_scope("Layer1"):
    W1 = tf.Variable(tf.random_normal([2, 2]), name="weight_1")
    b1 = tf.Variable(tf.random_normal([2]), name="bias_1")
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

    tf.summary.histogram("W1", W1)
    tf.summary.histogram("b1", b1)
    tf.summary.histogram("Layer1", layer1)


with tf.name_scope("Layer2"):
    W2 = tf.Variable(tf.random_normal([2, 1]), name="weight_2")
    b2 = tf.Variable(tf.random_normal([1]), name="bias_2")
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    tf.summary.histogram("W2", W2)
    tf.summary.histogram("b2", b2)
    tf.summary.histogram("Hypothesis", hypothesis)

# cost/loss function
with tf.name_scope("Cost"):
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
    tf.summary.scalar("Cost", cost)

with tf.name_scope("Train"):
    train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
tf.summary.scalar("accuracy", accuracy)

# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./logs/xor_logs
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs")
    writer.add_graph(sess.graph)  # Show the graph

    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        _, summary, cost_val = sess.run(
            [train, merged_summary, cost], feed_dict={X: x_data, Y: y_data}
        )
        writer.add_summary(summary, global_step=step)

        if step % 2000 == 0:
            print(step, cost_val)

    # Accuracy report
    h, p, a = sess.run(
        [hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data}
    )
    
    #print(f"\nHypothesis:\n{h} \nPredicted:\n{p} \nAccuracy:\n{a}")
    
    print('\n >Hypothesis :', h, 
          '\n >Correct :', p, 
          '\n >Accuracy :', a)

0 0.7323932
2000 0.34923428
4000 0.3471486
6000 0.34675288
8000 0.3466356
10000 0.34659585

 >Hypothesis : [[5.4453267e-05]
 [9.9997973e-01]
 [4.9999404e-01]
 [5.0000119e-01]] 
 >Correct : [[0.]
 [1.]
 [0.]
 [1.]] 
 >Accuracy : 0.5


### <font color='yellow' > Tensorboard 실행하기 </font>
- 1) 주피터, 아나콘다 종료 후 다시 아나콘다 cmd 실행 
- 2) 로그가 저장된 "D:\tensor_logs" 폴더로 이동
- 3) tensorboard --logdir=./ 실행 
- 4) 웹브라우저 주소창에 http://localhost:6006 입력 